In [ ]:
import requests
import pandas as pd
import time
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from functools import reduce
import streamlit as st

# Streamlit config
st.set_page_config(page_title="Token Tracker Dashboard", layout="wide")
st.title("📡 Token Analytics from XCAP API")

# Constants
BASE_URL = "https://xcap-mainnet.explorer.xcap.network/api/v2/token-transfers"
MAX_PAGES = 50
PAGE_DELAY = 0.3
FIELDS_TO_KEEP = [
    'transaction_hash',
    'token.symbol',
    'total.value',
    'from.hash',
    'to.hash',
    'timestamp',
    'token.exchange_rate',
    'type',
    'token.decimals'
]

@st.cache_data(ttl=28800)  # Cache for 8 hours
def fetch_all_token_transfers():
    page = 1
    all_data = []

    with st.spinner("Fetching data from XCAP API..."):
        while page <= MAX_PAGES:
            try:
                response = requests.get(BASE_URL, params={"page": page}, timeout=10)
                
                if response.status_code != 200:
                    st.warning(f"Error {response.status_code} on page {page}")
                    break

                data = response.json()
                items = data.get("items", [])

                if not items:
                    break

                all_data.extend(items)
                page += 1
                time.sleep(PAGE_DELAY)

            except requests.exceptions.RequestException as e:
                st.error(f"Request failed on page {page}: {e}")
                break

    df = pd.json_normalize(all_data)
    available_fields = [col for col in FIELDS_TO_KEEP if col in df.columns]
    return df[available_fields]

def process_data(df):
    # Clean and transform data
    df = df.drop_duplicates(subset=FIELDS_TO_KEEP)
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce').dt.tz_localize(None)
    
    # Convert numeric fields
    df['total.value'] = pd.to_numeric(df['total.value'], errors='coerce').fillna(0)
    df['token.decimals'] = pd.to_numeric(df['token.decimals'], errors='coerce').fillna(18)
    df['normalized_value'] = df['total.value'] / (10 ** df['token.decimals'])
    df['token.exchange_rate'] = pd.to_numeric(df['token.exchange_rate'], errors='coerce').fillna(0)
    df['usd_value'] = df['normalized_value'] * df['token.exchange_rate']
    
    return df

def calculate_metrics(df):
    # Calculate key metrics
    minted_total = df[df['type'] == 'token_minting']['normalized_value'].sum()
    burned_total = df[df['type'] == 'token_burning']['normalized_value'].sum()
    
    metrics = {
        "Metric": [
            "1. Total Asset Supply",
            "2. Unique Tokens",
            "3. Total Transactions",
            "4. Tokens Minted",
            "5. Tokens Burned",
            "6. Tokens Transferred",
            "7. Total Transaction Volume (USD)"
        ],
        "Value": [
            minted_total - burned_total,
            df['token.symbol'].nunique(),
            len(df),
            minted_total,
            burned_total,
            df[df['type'] == 'token_transfer']['normalized_value'].sum(),
            df['usd_value'].sum()
        ]
    }
    
    return pd.DataFrame(metrics)

def analyze_holdings(df):
    # Token holdings analysis
    tokens_sent = df[df['type'] == 'token_transfer'].groupby('from.hash')['normalized_value'].sum().reset_index()
    tokens_sent.columns = ['address', 'tokens_sent']
    
    tokens_received = df[df['type'] == 'token_transfer'].groupby('to.hash')['normalized_value'].sum().reset_index()
    tokens_received.columns = ['address', 'tokens_received']
    
    tokens_minted = df[df['type'] == 'token_minting'].groupby('to.hash')['normalized_value'].sum().reset_index()
    tokens_minted.columns = ['address', 'tokens_minted']
    
    tokens_burned = df[df['type'] == 'token_burning'].groupby('from.hash')['normalized_value'].sum().reset_index()
    tokens_burned.columns = ['address', 'tokens_burned']
    
    # Combine all data
    dfs = [tokens_minted, tokens_burned, tokens_received, tokens_sent]
    df_combined = reduce(lambda left, right: pd.merge(left, right, on='address', how='outer'), dfs).fillna(0)
    df_combined['token_holding'] = (df_combined['tokens_minted'] - df_combined['tokens_burned']) + \
                                  (df_combined['tokens_received'] - df_combined['tokens_sent'])
    
    # Calculate percentages
    total_token_holding = df_combined['token_holding'].sum()
    total_tokens_sent = df_combined['tokens_sent'].sum()
    total_tokens_received = df_combined['tokens_received'].sum()
    
    top10_holdings = df_combined.sort_values('token_holding', ascending=False).head(10)
    top10_holdings['% of Total Holding'] = (top10_holdings['token_holding'] / total_token_holding * 100).round(2)
    
    top10_sent = df_combined.sort_values('tokens_sent', ascending=False).head(10)[['address', 'tokens_sent']]
    top10_sent['% of Total Sent'] = (top10_sent['tokens_sent'] / total_tokens_sent * 100).round(2)
    
    top10_received = df_combined.sort_values('tokens_received', ascending=False).head(10)[['address', 'tokens_received']]
    top10_received['% of Total Received'] = (top10_received['tokens_received'] / total_tokens_received * 100).round(2)
    
    # Merge summaries
    summary_report = pd.merge(top10_holdings, top10_sent, on='address', how='outer')
    summary_report = pd.merge(summary_report, top10_received, on='address', how='outer')
    
    return summary_report.rename(columns={
        'token_holding': 'Token Holding',
        'tokens_sent': 'Tokens Sent',
        'tokens_received': 'Tokens Received'
    })

def analyze_trends(df):
    # Daily volume
    volume_per_day = df.groupby([df['timestamp'].dt.date, 'token.symbol'])['normalized_value'].sum().reset_index()
    volume_per_day.columns = ['date', 'token', 'daily_volume']
    
    # Cumulative supply
    df_mint = df[df['type'] == 'token_minting'].copy()
    df_burn = df[df['type'] == 'token_burning'].copy()
    
    df_mint['minted'] = df_mint['normalized_value']
    df_burn['burned'] = df_burn['normalized_value']
    
    mint_burn = pd.concat([df_mint[['timestamp', 'token.symbol', 'minted']], 
                         df_burn[['timestamp', 'token.symbol', 'burned']]], sort=False).fillna(0)
    mint_burn['date'] = mint_burn['timestamp'].dt.date
    
    supply = mint_burn.groupby(['date', 'token.symbol']).agg({'minted': 'sum', 'burned': 'sum'}).reset_index()
    supply['net_minted'] = supply['minted'] - supply['burned']
    supply['cumulative_supply'] = supply.groupby('token.symbol')['net_minted'].cumsum()
    
    # Top traded tokens
    traded_volume = df[df['type'] == 'token_transfer'].groupby('token.symbol')['normalized_value'].sum().reset_index()
    traded_volume.columns = ['token.symbol', 'total_transferred']
    
    return volume_per_day, supply, traded_volume

def create_plots(summary_report, volume_per_day, supply, top_tokens):
    # Helper function for horizontal bar charts
    def labeled_barh(data, column, title, color):
        fig, ax = plt.subplots(figsize=(4, 3))
        bars = ax.barh(data['address'], data[column], color=color)
        ax.set_xlabel(column)
        ax.set_title(title, fontsize=10)
        ax.tick_params(labelsize=8)
        ax.invert_yaxis()
        for bar in bars:
            ax.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, f'{bar.get_width():.2f}',
                    va='center', ha='left', fontsize=8)
        return fig
    
    # Create plots
    fig1 = labeled_barh(summary_report, "Token Holding", "Token Holdings", "green")
    fig2 = labeled_barh(summary_report, "Tokens Sent", "Tokens Sent", "red")
    fig3 = labeled_barh(summary_report, "Tokens Received", "Tokens Received", "blue")
    
    # Daily volume plot
    fig4, ax4 = plt.subplots(figsize=(6, 3))
    for token in volume_per_day['token'].unique():
        token_df = volume_per_day[volume_per_day['token'] == token]
        ax4.plot(token_df['date'], token_df['daily_volume'], marker='o', label=token)
        for x, y in zip(token_df['date'], token_df['daily_volume']):
            ax4.text(x, y, f"{y:.2f}", fontsize=7)
    ax4.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    ax4.tick_params(axis='x', labelrotation=45, labelsize=8)
    ax4.legend(fontsize=7)
    ax4.set_title("Daily Token Transfer Volume")
    
    # Cumulative supply plot
    fig5, ax5 = plt.subplots(figsize=(6, 3))
    for token in supply['token.symbol'].unique():
        token_df = supply[supply['token.symbol'] == token]
        ax5.plot(token_df['date'], token_df['cumulative_supply'], marker='o', label=token)
        for x, y in zip(token_df['date'], token_df['cumulative_supply']):
            ax5.text(x, y, f"{y:.2f}", fontsize=7)
    ax5.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    ax5.tick_params(axis='x', labelrotation=45, labelsize=8)
    ax5.legend(fontsize=7)
    ax5.set_title("Cumulative Token Supply")
    
    # Top tokens plot
    fig6, ax6 = plt.subplots(figsize=(6, 3))
    bars = ax6.barh(top_tokens['token.symbol'], top_tokens['total_transferred'], color='orange')
    ax6.set_xlabel("Total Transferred")
    ax6.set_ylabel("Token")
    ax6.tick_params(labelsize=8)
    ax6.invert_yaxis()
    for bar in bars:
        ax6.text(bar.get_width(), bar.get_y() + bar.get_height()/2, f"{bar.get_width():.2f}", 
                fontsize=8, va='center', ha='left')
    ax6.set_title("Most Transferred Tokens")
    
    return fig1, fig2, fig3, fig4, fig5, fig6

def main():
    # Load and process data
    raw_data = fetch_all_token_transfers()
    processed_data = process_data(raw_data)
    metrics_df = calculate_metrics(processed_data)
    summary_report = analyze_holdings(processed_data)
    volume_per_day, supply, top_tokens = analyze_trends(processed_data)
    
    # Create plots
    fig1, fig2, fig3, fig4, fig5, fig6 = create_plots(summary_report, volume_per_day, supply, top_tokens)
    
    # Dashboard layout
    st.title("📊 Token Distribution & Blockchain Trend Dashboard")
    
    # Sidebar filters
    st.sidebar.header("🔍 Filter Data")
    all_tokens = processed_data['token.symbol'].dropna().unique().tolist()
    all_types = processed_data['type'].dropna().unique().tolist()
    min_date = processed_data['timestamp'].min().date()
    max_date = processed_data['timestamp'].max().date()
    
    selected_token = st.sidebar.selectbox("Select Token", ["All"] + all_tokens)
    selected_type = st.sidebar.selectbox("Select Transaction Type", ["All"] + all_types)
    date_range = st.sidebar.date_input("Select Date Range", [min_date, max_date])
    
    # Filter data based on selection
    df_filtered = processed_data.copy()
    if selected_token != "All":
        df_filtered = df_filtered[df_filtered['token.symbol'] == selected_token]
    if selected_type != "All":
        df_filtered = df_filtered[df_filtered['type'] == selected_type]
    if len(date_range) == 2:
        df_filtered = df_filtered[
            (df_filtered['timestamp'].dt.date >= date_range[0]) & 
            (df_filtered['timestamp'].dt.date <= date_range[1])
    
    # Filter trend data
    df_trend_filtered = volume_per_day.copy()
    df_supply_filtered = supply.copy()
    df_top_filtered = top_tokens.copy()
    
    if selected_token != "All":
        df_trend_filtered = df_trend_filtered[df_trend_filtered['token'] == selected_token]
        df_supply_filtered = df_supply_filtered[df_supply_filtered['token.symbol'] == selected_token]
    
    # Metrics section
    st.subheader("📊 Key Metrics")
    st.dataframe(metrics_df.style.format({"Value": "{:,.2f}"}))
    
    # Summary section
    st.subheader("📦 Top Token Holders and Distribution Summary")
    st.dataframe(summary_report.style.format({
        "Token Holding": "{:.6f}",
        "% of Total Holding": "{:.2f}%",
        "Tokens Sent": "{:.6f}",
        "% of Total Sent": "{:.2f}%",
        "Tokens Received": "{:.6f}",
        "% of Total Received": "{:.2f}%"
    }))
    
    # Holdings section
    st.subheader("📈 Visual Breakdown by Address")
    col1, col2, col3 = st.columns(3)
    with col1:
        st.pyplot(fig1)
    with col2:
        st.pyplot(fig2)
    with col3:
        st.pyplot(fig3)
    
    # Trends section
    st.subheader("📉 Blockchain Activity Trends")
    col4, col5 = st.columns(2)
    with col4:
        st.markdown("##### 🔄 Daily Token Transfer Volume")
        st.pyplot(fig4)
    with col5:
        st.markdown("##### 📈 Cumulative Token Supply")
        st.pyplot(fig5)
    
    # Top tokens section
    st.subheader("🏆 Most Transferred Tokens")
    st.pyplot(fig6)
    
    # Raw data section
    st.subheader("📄 Cleaned Token Transfer Records")
    st.dataframe(df_filtered.reset_index(drop=True))
    
    st.markdown("---")
    st.markdown("Made with ❤️ by Qenehelo Matjama")

if __name__ == "__main__":
    main()